Notebook to the plot reparameterization described in Appendix A

In [ ]:
import matplotlib.pyplot as plt
from bayesbeat.config import read_config
from nessai.reparameterisations import default_reparameterisations
from nessai.livepoint import empty_structured_array
import h5py
import pathlib

plt.style.use("paper.mplstyle")

In [ ]:
outdir = pathlib.Path("figures") / "reparameterisations"
outdir.mkdir(parents=True, exist_ok=True)

In [ ]:
result_file = pathlib.Path("../results/bayesbeat_inference_results/simulated_data/model_3/result_ringdown_1.hdf5")

with h5py.File(result_file, 'r') as f:
    posterior = f["posterior_samples"][:]


In [ ]:
config_file = pathlib.Path("../analysis/simulated_data") / "model_3.ini"
config = read_config(config_file)

## Apply the reparameterization

This uses code from `nessai` to construct the class and apply the reparameterization.

In practice, this is done automatically within `nessai`.

In [ ]:
prior_bounds = config["Model"]["prior_bounds"]

In [ ]:
reparam = default_reparameterisations["log_amplitude"].class_fn(
    parameters=["log10_a_1", "log10_a_scale"],
    prior_bounds={p: prior_bounds[p] for p in ["log10_a_1", "log10_a_scale"]},
)

In [ ]:
x = posterior.copy()
x_prime = empty_structured_array(len(x), reparam.prime_parameters)
log_j = 0
reparam.update(x)
x, x_prime, log_j = reparam.reparameterise(x, x_prime, log_j)

## Figure A1

In [ ]:
figsize = plt.rcParams["figure.figsize"].copy()
figsize[0] *= 1.8
fig, axs = plt.subplots(1, 2, figsize=figsize)

a_1 = 10 ** posterior["log10_a_1"]
a_scale = 10 ** posterior["log10_a_scale"]
colour = posterior["a_ratio"]

axs[0].scatter(a_1, a_scale, s=1, c=colour, rasterized=True)
axs[0].set_xlabel(r"$A_1$")
axs[0].set_ylabel(r"$A_\text{scale}$")

scatter = axs[1].scatter(x_prime["alpha"], x_prime["beta"], c=colour, s=1, rasterized=True)
axs[1].set_xlabel(r"$\alpha$")
axs[1].set_ylabel(r"$\beta$")

# plt.tight_layout()

plt.subplots_adjust(wspace=0.3)

fig.colorbar(scatter, ax=axs.ravel().tolist(), label=r"$\rho$")

fig.savefig(outdir / "reparameterization.pdf")
plt.show()